### Input Data
$N$: List of nodes

$D_{ij}$: Distance between node $i$ and hub $j$

$g_j$: Distance between hub $j$ and gateway hub SFA 

$d_i$: Customer demand of node $i$

### Decision variables

$x_{ij} = \mathbb{1}\{\text{if node } i \text{ is assigned to node } j \}$

$y_{j} = \mathbb{1}\{\text{if node } j \text{ is a local hub}  \}$

$C_i$: Number of Type C Vans needed for Node $i$

$A_j$: Number of Type A Vans needed for Hub $j$

$B1_i$: Number of Type B Vans needed for Node $i$

$B2_j$: Number of Type B Vans needed for Hub $j$

### Constraints
 - Node i will only be assigned to node j if j is a local hub.
 - Each node is only assigned to one local hub.
$$
x_{ij} <= y_{j} \ \text{ for all }  i, j \in N \\
\sum_{j \in N} x_{ij} = 1 \ \text{ for each node } i \in N \\
$$

 - Number of vehicles can fulfill the demand.
$$ 40 C_i + 200 B1_i>= d_i \text{ for each node } i \in N $$
$$ 800 A_j + 200 B2_j >= \sum_{i \in N} x_{ij} \cdot d_i \text{ for each hub } j \in N$$

### Cost Decomposition
1. Fixed Cost
 - Unit cost: 20
$$F \cdot \sum_{j \in N}y_{j}, F=20$$
2. Cost of Type C Vans
 - Unit cost: 6 * dist
$$ \sum x_{ij} \cdot 6 D_{ij} \cdot C_i$$
3. Cost of Type A Vans 
 - Unit cost: max(70, 70+ 4.5 * (dist - 5))
$$ \sum y_j \cdot max(70, 70+4.5(g_{j}-5)) \cdot A_j$$
$$ g'_{j} = max(5, g_{j})$$
$$ \sum y_j \cdot (70+4.5(g'_{j}-5)) \cdot A_j$$
4. Cost of Type B2 Vans
 - Unit cost: max(30, 30+ 4 * (dist - 5))
$$ \sum y_j \cdot (30+4(g'_{j}-5)) \cdot B2_j$$
5. Cost of Type B1 Vans
 - Unit cost: max(30, 30+ 4 * (dist - 5))
$$ \sum x_{ij} \cdot (30+4(D'_{ij}-5)) \cdot B1_i$$

### Objective Function
$$\text{minimize} \quad 20 \sum_{j \in N}y_{j} + \sum_{i,j \in N}x_{ij} \cdot 6 D_{ij} \cdot C_i + \sum_{j \in N}y_j \cdot (70+4.5(g'_{j}-5)) \cdot A_j + \sum y_j \cdot (30+4(g'_{j}-5)) \cdot B2_j + \sum x_{ij} \cdot (30+4(D'_{ij}-5)) \cdot B1_i$$

In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np

## Data Preparation

In [2]:
D = pd.read_csv("Data\d_matrix.csv", index_col = 0)
demand = pd.read_excel("Data\demand.xlsx", index_col = 0)
d = demand.demand

In [3]:
D2 = pd.DataFrame()
for i in D.columns:
    D2[i] = D[i].apply(lambda x: max(x, 5) if x>0 else  (-30/4 + 5))

In [4]:
# g: distance with SFA
adist = D.iloc[-2]

# convert g to g'
g = adist.apply(lambda x: max(5, x))

In [16]:
import os

path = 'Data/Data_demand & distance_clusters/demand'   
files = os.listdir(path) 

df1 = pd.read_csv(path + '/' + files[0])
df1['name'] = files[0].split('.')[0].split('_')[1]

for file in files[1:]:     
    df2 = pd.read_csv(path +'/' +  file) 
    df2['name'] = file.split('.')[0].split('_')[1]
    df1 = pd.concat([df1,df2],axis=0,ignore_index=True) 

df1 = df1.drop_duplicates()  
df1 = df1.reset_index(drop=True) 

In [18]:
df1.to_csv('Data/cluster_result20.csv',index = 0)

In [19]:
clusters = df1

In [21]:
cluster_name = list(clusters.name.unique())
cluster_name

['cluster1',
 'cluster10',
 'cluster11',
 'cluster12',
 'cluster13',
 'cluster14',
 'cluster15',
 'cluster16',
 'cluster17',
 'cluster18',
 'cluster19',
 'cluster2',
 'cluster20',
 'cluster3',
 'cluster4',
 'cluster5',
 'cluster6',
 'cluster7',
 'cluster8',
 'cluster9']

In [22]:
# number of nodes in every clusters
clusters['name'].value_counts()

cluster1     231
cluster16    160
cluster6     134
cluster17    132
cluster12    120
cluster19    109
cluster10    104
cluster2     103
cluster15    103
cluster14     93
cluster3      90
cluster4      85
cluster20     84
cluster8      79
cluster9      78
cluster11     76
cluster18     68
cluster5      66
cluster7      58
cluster13     48
Name: name, dtype: int64

### Initialization

In [27]:
basemodel = pd.read_csv(r'Data/basemodel.csv',index_col = 0)
basemodel = basemodel.fillna(0)
basemodel

,SF0001,SF0002,SF0003,SF0004,SF0005,SF0006,SF0007,SF0008,SF0009,SF0010,...,SF2012,SF2013,SF2014,SF2015,SF2016,SF2017,SF2018,SF2019,SF2020,SF2021
customer_code,,,,,,,,,,,,,,,,,,,,,
SF0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SF0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SF0003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SF0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SF0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
SF2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
SF2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [61]:
hubs = basemodel.sum()
hubs = hubs.apply(lambda x: 1 if x > 0 else 0)

### Get Hub for each Node

In [79]:
hub_list = basemodel.idxmax(1)

In [80]:
hub_list.to_csv("hub_list.csv")

### Model

In [41]:
def gsolver(node):
    model1 = pe.ConcreteModel()
    
    def ini1(mod, i, j):
        mod.x[i,j] = basemodel.loc[i,j]
        return mod.x[i,j]
    def ini2(mod, j):
        mod.y[j] = hubs[j]
        return mod.y[j]
    model1.x = pe.Var(node, node, domain = pe.Binary, initialize = ini1)
    model1.y = pe.Var(node, domain = pe.Binary, initialize = ini2)
    model1.a = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.c = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b1 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b2 = pe.Var(node, domain=pe.NonNegativeIntegers)
    
    

    model1.costs = pe.Objective(
        expr = 20 * sum(model1.y[j] for j in node) +
        sum(model1.x[i,j] * 6 * D.loc[i,j] * model1.c[i] for i in node for j in node) + 
        sum(model1.x[i,j] * (30+4*(D2.loc[i,j]-5)) * model1.b1[i] for i in node for j in node) + 
        sum(model1.y[j] * (70+4.5*(g[j]-5)) * model1.a[j] for j in node) +
        sum(model1.y[j] * (30+4*(g[j]-5)) * model1.b2[j] for j in node),
        sense = pe.minimize)

    # A node can only be assigned to a hub node
    def rule_1(mod, i, j):
        return mod.x[i,j] <= mod.y[j]
    model1.const1 = pe.Constraint(node, node, rule = rule_1)

    # Each node is assigned to 1 and only 1 hub
    def rule_2(mod, i):
        return sum(mod.x[i, j] for j in node) == 1
    model1.const2 = pe.Constraint(node, rule = rule_2)

    # Number of Vans for terminal delivery
    def rule_3(mod, i):
        return 40 * mod.c[i] + 200 * mod.b1[i] >= d[i]
    model1.const3 = pe.Constraint(node, rule = rule_3)

    # Number of Vans for distribution
    def rule_4(mod, j):
        return 800 * mod.a[j] + 200 * mod.b2[j] >= sum(mod.x[i,j] * d[i] for i in node)
    model1.const4 = pe.Constraint(node, rule = rule_4)

    time_limit = 10*60
    solver = pe.SolverFactory('gurobi')
    solver.options['TimeLimit'] = time_limit
    result = solver.solve(model1, tee = True, warmstart = True)
    
    return model1

In [58]:
def result(cluster, node, model1):
    c = model1.costs()
    costs[cluster] = round(c,2)
    
    hub = []
    avan = []
    bvan = []
    for j in node:
        if round(model1.y[j]()) == 1:
            hub.append(j)
            avan.append(model1.a[j]())
            bvan.append(model1.b2[j]())
    local = pd.DataFrame({"cluster":cluster, "hub":hub, "TypeA":avan, "TypeB":bvan})
    
    df = pd.DataFrame(columns = node, index = node)
    for i in node:
        for j in node:
            df.loc[i][j] = round(model1.x[i,j]())
    
    local.to_csv('20abc/' + cluster + "result20bc_hub.csv", index = 0)
    df.to_csv('20abc/' + cluster + "result20bc_node.csv")
    
    return c, local, df

In [59]:
clus = cluster_name[1]
node = list(clusters[clusters.name == clus]['customer_code'])

In [63]:
%%time
costs = {}
for clus in cluster_name[1:]:
    node = list(clusters[clusters.name == clus]['customer_code'])
    model1 = gsolver(node)
    c, local, df = result(clus, node, model1)
    print(clus, round(c, 2))

Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpavckq7xj.pyomo.lp
Reading time = 0.09 seconds
x11337: 11129 rows, 11337 columns, 43681 nonzeros
Read MIP start from file C:\Users\Yishu\AppData\Local\Temp\tmpf5g80ckf.gurobi.mst
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 11129 rows, 11337 columns and 43681 nonzeros
Model fingerprint: 0xc788dfcc
Model has 21628 quadratic objective terms
Variable types: 1 continuous, 11336 integer (10920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+01, 2e+01]
  QObjective range [5e-02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

User MIP start produce

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 6005 rows, 6157 columns and 23409 nonzeros
Model fingerprint: 0xcfff73f4
Model has 11552 quadratic objective terms
Variable types: 1 continuous, 6156 integer (5852 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+01, 2e+01]
  QObjective range [1e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

User MIP start produced solution with objective 2785.36 (0.01s)
Loaded user MIP start with objective 2785.36

Presolve removed 1 rows and 1 columns
Presolve time: 0.06s
Presolved: 29108 rows, 40812 columns, 81168 nonzeros
Presolved model has 23104 SOS constraint(s)
Variable types: 0 continuous, 40812 integer (17404 binary)

Root relaxation: objective 2.000000e+01, 16544 iterations, 0.15 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

 49863 43490 1537.88029  163  331 2783.10099 1129.29852  59.4%  64.0  495s
 51064 44965 1188.50501   33   77 2783.10099 1129.31533  59.4%  64.5  504s
 52003 46340 1779.54910  218   65 2783.10099 1129.51783  59.4%  64.0  510s
 54599 47773 1366.28002  126   74 2783.10099 1129.63990  59.4%  62.6  518s
 55341 48689 1276.84410   45   77 2783.10099 1129.75551  59.4%  62.7  523s
 56356 49213 1359.98459   36   80 2783.10099 1129.84938  59.4%  62.1  526s
 57056 49797 1155.19372   29  297 2783.10099 1129.95010  59.4%  62.3  531s
 57788 50389 1328.66291  102   74 2783.10099 1129.95010  59.4%  62.2  535s
 58435 50822 2204.61108  229   61 2783.10099 1130.01820  59.4%  62.1  540s
 59552 52204 1310.99592   85  291 2783.10099 1130.01820  59.4%  62.3  549s
 60441 52774 1560.85639  106   81 2783.10099 1130.01820  59.4%  62.1  553s
 61043 53335 1816.61981  160  272 2783.10099 1130.01820  59.4%  62.2  557s
 61735 54003 1131.84747   38  448 2783.10099 1130.13044  59.4%  62.3  561s
 62447 54725 1139.44217  

    containing a solution
cluster12 3107.8
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpgaicfj_m.pyomo.lp
Reading time = 0.03 seconds
x2545: 2449 rows, 2545 columns, 9409 nonzeros
Read MIP start from file C:\Users\Yishu\AppData\Local\Temp\tmpjy5opv8z.gurobi.mst
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2449 rows, 2545 columns and 9409 nonzeros
Model fingerprint: 0x7159a2ab
Model has 4608 quadratic objective terms
Variable types: 1 continuous, 2544 integer (2352 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+01, 2e+01]
  QObjective range [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+0

 157992 132994 1296.46050   77   41 1304.96037  514.63353  60.6%  42.4  370s
 160821 135347  627.74550   41   53 1304.96037  515.25478  60.5%  42.1  376s
 162591 136740  721.06148   90   68 1304.96037  515.48772  60.5%  41.9  380s
 164342 138549  893.29136  159   58 1304.96037  515.56650  60.5%  41.8  385s
 167104 140668  765.79000  127   47 1304.96037  515.64455  60.5%  41.6  390s
 168624 141858  936.29416  154   49 1304.96037  515.70773  60.5%  41.5  395s
 170936 143602  981.32802  101  170 1304.96037  515.99494  60.5%  41.4  400s
 172883 145368  536.82289   78   52 1304.96037  515.99494  60.5%  41.3  406s
 174656 146946  551.21136   41  186 1304.96037  516.30267  60.4%  41.1  410s
 177154 149061  581.50293   76   52 1304.96037  516.32474  60.4%  40.9  416s
 178773 150352 1009.39850  112   44 1304.96037  516.35902  60.4%  40.8  420s
 181163 152422  709.69310   76   50 1304.96037  516.62007  60.4%  40.7  426s
 182878 153757  891.45880  125   58 1304.96037  516.75612  60.4%  40.6  430s

...

Solution count 4: 2050.9 2050.9 2055.77 2062.25 

Time limit reached
Best objective 2.050899000000e+03, best bound 8.366526052261e+02, gap 59.2056%
    containing a solution
cluster5 2050.9
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpk_wmnp9j.pyomo.lp
Reading time = 0.15 seconds
x18627: 18359 rows, 18627 columns, 72361 nonzeros
Read MIP start from file C:\Users\Yishu\AppData\Local\Temp\tmpwd69f8jy.gurobi.mst
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 18359 rows, 18627 columns and 72361 nonzeros
Model fingerprint: 0xe96b2a2f
Model has 35912 quadratic objective terms
Variable types: 1 continuous, 18626 integer (18090 binary)
Coefficient statistics:
  M

  Objective range  [2e+01, 2e+01]
  QObjective range [3e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]

User MIP start produced solution with objective 1636.65 (0.00s)
Loaded user MIP start with objective 1636.65

Presolve removed 1 rows and 1 columns
Presolve time: 0.03s
Presolved: 16994 rows, 23838 columns, 47328 nonzeros
Presolved model has 13456 SOS constraint(s)
Variable types: 0 continuous, 23838 integer (10150 binary)

Root relaxation: objective 2.000000e+01, 9406 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   20.00000    0   60 1636.65150   20.00000  98.8%     -    0s
     0     0   20.00000    0   58 1636.65150   20.00000  98.8%     -    0s
     0     0   20.00000    0   58 1636.65150   20.00000  98.8%     -    1s
     0     2   20.00000    0   58 1636.65150   20.00000  98.8%     -    2s
   235   258  

H   71    78                    1604.1875000   20.00000  98.8%   794   11s
   161   183   20.00000   13   81 1604.18750   20.00000  98.8%   667   15s
   403   434   20.00000   23   81 1604.18750   20.00000  98.8%   433   20s
   754   786   20.00000   37   81 1604.18750   20.00000  98.8%   303   25s
  1120  1154   40.00000   49   81 1604.18750   20.00000  98.8%   253   30s
  1571  1642   40.00000   65   81 1604.18750   20.00000  98.8%   231   35s
  2371  2612   60.00000   78   81 1604.18750   40.00000  97.5%   182   42s
  2886  2613   80.00000   27    0 1604.18750   40.00000  97.5%   163   45s
  2891  2619  415.55200   13  317 1604.18750  415.55200  74.1%   166   53s
  2893  2623  415.55200   14   79 1604.18750  415.55200  74.1%   166   57s
  2897  2629  415.55200   15  317 1604.18750  415.55200  74.1%   169   61s
  2913  2640  438.26270   16  313 1604.18750  415.55200  74.1%   174   67s
  2937  2664  415.55200   18   79 1604.18750  415.55200  74.1%   175   72s
  2983  2698  469.41250  

 81809 73331  536.23021  150  313 1604.18750  435.41250  72.9%  40.0  578s
 83371 74551  902.03400  218   73 1604.18750  435.41250  72.9%  39.5  585s
 84681 75746  504.52615   67   82 1604.18750  435.41250  72.9%  39.0  593s
 85923 76402 1358.30400  249   66 1604.18750  435.41250  72.9%  38.6  598s
 86720 76549  442.54161   54  149 1604.18750  435.41250  72.9%  38.5  600s

Explored 86889 nodes (3365632 simplex iterations) in 600.28 seconds
Thread count was 16 (of 16 available processors)

Solution count 6: 1604.19 1604.19 1604.19 ... 1616.28

Time limit reached
Best objective 1.604187500000e+03, best bound 4.354125000000e+02, gap 72.8578%
    containing a solution
cluster8 1604.19
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpfxzuukx0.pyomo.lp
Reading time = 0.06 seconds
x6475: 6319 rows, 6475 columns, 24649 nonzeros
Read MIP start from file C:\Users\Yi

 30053 25683  983.17244  157   80 2300.52850  641.41099  72.1%  88.3  386s
 31169 26643  698.64202   34   82 2300.52850  641.41099  72.1%  87.5  391s
 32260 27232  708.64429   53   84 2300.52850  641.41099  72.1%  86.7  397s
 32849 28693  756.04691   90   83 2300.52850  641.41099  72.1%  87.6  403s
 34449 29307  753.58146  119   82 2300.52850  641.41099  72.1%  85.3  410s
 35128 29933  817.20250  134   75 2300.52850  641.41099  72.1%  85.9  415s
 35799 30662  877.91050  146   73 2300.52850  641.41099  72.1%  86.1  420s
 36551 31292 1347.35450  232   68 2300.52850  642.30609  72.1%  86.3  425s
 37275 32748  666.76638   22   80 2300.52850  642.30609  72.1%  86.6  432s
 38883 33410  824.09822   65   83 2300.52850  642.30609  72.1%  84.5  437s
 39603 34212 1068.71862  157   75 2300.52850  642.40619  72.1%  84.2  442s
 40467 35671  825.19622   66   82 2300.52850  642.40619  72.1%  84.1  448s
 42111 36388 1059.76574  162   76 2300.52850  642.79959  72.1%  82.3  453s
H42336 35885             

In [2]:
costs['cluster1'] = 4621.3 # calculated earlier

In [71]:
total_cost = 0
for k,v in costs.items():
    total_cost += v
total_cost

49503.810000000005

In [72]:
costs

{'cluster10': 2327.5,
 'cluster11': 2783.1,
 'cluster12': 3107.8,
 'cluster13': 1304.96,
 'cluster14': 1762.48,
 'cluster15': 2006.98,
 'cluster16': 3640.73,
 'cluster17': 3167.2,
 'cluster18': 1685.48,
 'cluster19': 2884.96,
 'cluster2': 2512.91,
 'cluster20': 1964.62,
 'cluster3': 2387.56,
 'cluster4': 2473.12,
 'cluster5': 2050.9,
 'cluster6': 3338.9,
 'cluster7': 1596.56,
 'cluster8': 1604.19,
 'cluster9': 2282.56,
 'cluster1': 4621.3}